<a href="https://colab.research.google.com/github/ChanakaDushmantha/Amazon-Review-Scraping/blob/main/Amazon_Selenium_colab_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install selenium
%pip install pandas
%pip install bs4

In [ ]:
from selenium.webdriver.common.by import By
import time
from selenium.webdriver import Firefox
import pandas as pd
import re
import traceback
from selenium import webdriver
from IPython.display import Image

In [ ]:
filename = 'amazon_urls.txt'
file = open('amazon_urls.txt')
LIST=re.split("\r\n|\n",file.read())
file.close()
x = []
for i in LIST:
    print(i.split('~')[-1])
    x.append(i.split('~')[-1])

In [ ]:
# -- to avoid captcha run amazon url two times and use same driver again and again--
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Firefox(options=options)

In [ ]:
# initial testing round
driver.get("https://www.amazon.com/Tomorrow-smash-hit-Sunday-Times-bestseller/dp/152911554X/ref=zg_bs_g_269678_sccl_1/260-6217822-9204433?psc=1")
display(Image(driver.get_full_page_screenshot_as_png()))

In [ ]:
reviews,reviewers, reviewtitles,review_date,reviewerplace,reviewratings, dfs = [],[],[],[],[],[],[]
product_names, prices, auther_names = [],[],[]

l = 0
while l < len(x):
#   url = 'https://www.amazon.com/Bumble-Building-Repair-Styling-Cream/dp/B08V8HH6JS'
    url = x[l]

    #extract product name from url
    try:
        driver.get(url)
        product_name = url.split("/")[3].replace("-"," ")
        print(product_name, end='\n')

    except Exception as ex:
        product_name = ""
        traceback.print_exc()

    #extract product price
    try:
        price_tag = driver.find_elements(By.CSS_SELECTOR, '#corePrice_feature_div')
        print(price_tag[0].text)
        price = price_tag[0].text

        if price_tag[0].text=='':
            price_tag1 = driver.find_elements(By.CSS_SELECTOR, '#newBooksSingleBuyingOptionHeader_feature_div')
            print(price_tag1[0].text.split("\n")[1])
            price = price_tag1[0].text.split("\n")[1]

    except Exception as ex:
        price = ""
        traceback.print_exc()
        display(Image(driver.get_full_page_screenshot_as_png()))

    #extract auther name
    try:
        auther = driver.find_elements(By.CSS_SELECTOR, '#bylineInfo')
        print(auther[0].text.replace("by ","").replace(" (Author)",""))
        auther_name = auther[0].text.replace("by ","").replace(" (Author)","")

    except Exception as ex:
        auther_name = ""
        traceback.print_exc()

    try:
        all_reviews = driver.find_element(By.CSS_SELECTOR, '#reviews-medley-footer .a-text-bold')
        all_reviews.click()
    except Exception as ex:
        traceback.print_exc()
        display(Image(driver.get_full_page_screenshot_as_png()))
        l+=1
        continue

    #scrape review data
    while True:
        #ratings = driver.find_elements(By.XPATH, '//i[@data-hook="review-star-rating"]/span')
        #lst_of_ratings = driver.find_elements(By.CLASS_NAME, "review-rating")
        #is_verified = driver.find_elements(By.XPATH, '//span[@data-hook="avp-badge"]')
        loc_date = driver.find_elements(By.XPATH, "//span[@data-hook='review-date']")

        rating1 = driver.find_elements(By.XPATH, "//i[@data-hook='review-star-rating']/span")
        rating2 = driver.find_elements(By.XPATH, "//i[@data-hook='cmps-review-star-rating']/span")
        rating = rating1 + rating2

        review_title1 = driver.find_elements(By.XPATH, "//a[@data-hook='review-title']/span[2]")
        review_title2 = driver.find_elements(By.XPATH, "//span[@data-hook='review-title']/span")
        review_title = review_title1 + review_title2

        review = driver.find_elements(By.XPATH, '//span[@data-hook="review-body"]')

        reviewer1 = driver.find_elements(By.CSS_SELECTOR, '#cm_cr-review_list .a-profile-name')
        reviewer = [rev.text.strip() for rev in reviewer1 if rev.text.strip()]

        max_value = max(len(loc_date), len(rating), len(review_title), len(review), len(reviewer))
        print(max_value)
        for i in range(max_value):
            try:
                other, date = loc_date[i].text.split('on')
                match = re.search(r'in\s+(.*?)\s+on', loc_date[i].text)
                place = match.group(1) if match else ""
            except:
                place, date = "", ""
                pass

            review_date.append(date)
            reviewerplace.append(place)

            try:
                reviewratings.append(rating[i].get_attribute('textContent'))
            except Exception as ex:
                traceback.print_exc()
                reviewratings.append("")

            try:
                reviewtitles.append(review_title[i].text)
            except Exception as ex:
                reviewtitles.append("")
                traceback.print_exc()

            try:
                reviews.append(review[i].text)
            except Exception as ex:
                reviews.append("")
                traceback.print_exc()

            try:
                reviewers.append(reviewer[i])
            except Exception as ex:
                reviewers.append("")
                traceback.print_exc()

            product_names.append(product_name)
            prices.append(price)
            auther_names.append(auther_name)

        try:
            next_btn = driver.find_element(By.XPATH, "//ul[@class='a-pagination']/li[2]/a")
            next_btn.click()
            print("goto next page")
            time.sleep(5)

        except Exception as ex:
            print("next page is not found")
            break

    l+=1

final_data = {'Product':product_names,'Auther':auther_names,'Price':prices,'Review Title':reviewtitles,'Reviews':reviews,'Reviewer':reviewers,
    'Date':review_date,'Place':reviewerplace,'Rating':reviewratings}
min_length = len(reviewtitles)
final_df = pd.DataFrame({k:pd.Series(v[:min_length]) for k,v in final_data.items()})
final_df.to_csv('Amazon_review_data.csv',index=False)
del final_df
# driver.quit()

print("Finished...")

In [ ]:
# script for merge csv files
import pandas as pd
import os

# Set the directory to the current working directory
csv_directory = '/content'

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(csv_directory) if file.endswith('.csv')]

# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Loop through each CSV file and merge the data
for csv_file in csv_files:
    file_path = os.path.join(csv_directory, csv_file)
    df = pd.read_csv(file_path, encoding='latin-1')

    # Skip merging headers for all files except the first one
    if not merged_data.empty:
        df = df[1:]

    merged_data = pd.concat([merged_data, df], ignore_index=True)

# Write the merged data to a new CSV file
merged_data.to_csv('/content/Amazon_review_merged_data.csv', index=False)